In [34]:
from __future__ import annotations

from pathlib import Path
import numpy as np
import pandas as pd
import yaml

CONFIG_PATH = Path("ammonia_config.yaml")

with CONFIG_PATH.open("r", encoding="utf-8") as f:
    CFG = yaml.safe_load(f)

CFG.keys()

dict_keys(['years', 'input', 'output', 'constants', 'maritime', 'plateau', 'scenarios'])

In [35]:
YEAR_START = int(CFG["years"]["start"])
YEAR_END = int(CFG["years"]["end"])
YEARS = np.arange(YEAR_START, YEAR_END + 1)

INPUT_CSV = Path(CFG["input"]["csv_path"])
BASE_YEAR = int(CFG["input"]["base_year"])

OUT_BASE = Path(CFG["output"]["base_dir"])
SCENARIO_DIRS = {k: OUT_BASE / v for k, v in CFG["output"]["scenario_dirs"].items()}

CST = CFG["constants"]
HB_KWH_PER_T = float(CST["hb_electricity_kwh_per_t_nh3"])
H2_T_PER_T = float(CST["h2_t_per_t_nh3"])

SMR_STOICH = float(CST["smr_stoich_ch4_t_per_t_h2"])
SMR_EFF = float(CST["smr_h2_efficiency"])
if not (0 < SMR_EFF <= 1.0):
    raise ValueError("smr_h2_efficiency doit être dans (0,1].")

CH4_T_PER_T_H2 = SMR_STOICH / SMR_EFF

STEP_YEARS = int(CFG.get("plateau", {}).get("step_years", 5))
if STEP_YEARS <= 0:
    raise ValueError("plateau.step_years doit être > 0.")

SC_PARAMS = CFG["scenarios"]

YEAR_START, YEAR_END, STEP_YEARS, CH4_T_PER_T_H2

(2019, 2050, 5, 2.6315789473684212)

In [36]:
MAR = CFG.get("maritime", {})
MAR_ENABLED = bool(MAR.get("enabled", False))

MAR_BASE_2019_T = float(MAR.get("base_2019_t", 0.0))
MAR_CAGR = float(MAR.get("cagr", 0.0))
MAR_ALLOC = str(MAR.get("allocation", "proportional_to_base")).strip()

if MAR_CAGR < -0.99:
    raise ValueError("maritime.cagr trop négatif (doit être >= -0.99).")

if MAR_ALLOC not in {"proportional_to_base", "equal_by_country"}:
    raise ValueError("maritime.allocation doit être 'proportional_to_base' ou 'equal_by_country'.")

In [37]:
def plateau_series(
    years: np.ndarray,
    start_year: int,
    end_year: int,
    start_value: float,
    end_value: float,
    step_years: int,
) -> np.ndarray:
    """
    Série en escaliers (plateaux) entre start_value et end_value.
    La valeur évolue uniquement aux bornes de blocs de longueur step_years.

    Exemple :
    - start_year=2020, end_year=2050, step=5
    - blocs : [2020-2024], [2025-2029], ..., [2050-2050]
    - valeur constante par bloc, progressant linéairement d'un bloc à l'autre.
    """
    y = years.astype(int)
    out = np.empty_like(years, dtype=float)

    # Années avant start -> start_value ; après end -> end_value
    out[y < start_year] = float(start_value)
    out[y >= end_year] = float(end_value)

    # Intervalle à traiter
    mask = (y >= start_year) & (y < end_year)
    yy = y[mask]

    # index de bloc
    block_idx = (yy - start_year) // step_years

    # nombre de blocs
    n_blocks = int(np.ceil((end_year - start_year) / step_years))
    n_blocks = max(n_blocks, 1)

    # fraction d'avancement par bloc (0 .. 1)
    frac = block_idx / n_blocks
    frac = np.clip(frac, 0.0, 1.0)

    out[mask] = float(start_value) + frac * (float(end_value) - float(start_value))
    return out


def assert_non_negative(df: pd.DataFrame, cols: list[str]) -> None:
    cols_present = [c for c in cols if c in df.columns]
    if not cols_present:
        return
    if (df[cols_present] < -1e-9).any(axis=None):
        mins = df[cols_present].min()
        raise ValueError(f"Valeurs négatives détectées:\n{mins}")


def check_coverage(df: pd.DataFrame, y0: int, y1: int) -> None:
    keys = ["country", "base_scenario", "scenario"]
    for k, g in df.groupby(keys):
        ys = np.sort(g["year"].unique())
        expected_n = (y1 - y0 + 1)
        if ys[0] != y0 or ys[-1] != y1 or len(ys) != expected_n:
            raise ValueError(
                f"Couverture années KO pour {k}: "
                f"{ys[0]}..{ys[-1]} n={len(ys)} (attendu {expected_n})."
            )

In [38]:
# CSV attendu : 4 colonnes
# - soit SANS en-tête : AT,DE,2019,455000.0
# - soit AVEC en-tête : country,base_scenario,year,nh3_consumption_t

def read_ammonia_raw(path: Path) -> pd.DataFrame:
    # Tentative 1 : lecture "avec header"
    df = pd.read_csv(path)

    # Si le fichier n'a pas de header, pandas créera des noms 0..3
    # On détecte et on relit en mode header=None.
    if not {"country", "base_scenario", "year", "nh3_consumption_t"}.issubset(df.columns):
        df = pd.read_csv(
            path,
            header=None,
            names=["country", "base_scenario", "year", "nh3_consumption_t"],
        )

    # Normalisation minimale + conversions robustes
    df["country"] = df["country"].astype(str).str.strip().str.upper()
    df["base_scenario"] = df["base_scenario"].astype(str).str.strip()

    # Conversion robuste : si une ligne "year" traîne encore, elle devient NaN puis on drop
    df["year"] = pd.to_numeric(df["year"], errors="coerce")
    df["nh3_consumption_t"] = pd.to_numeric(df["nh3_consumption_t"], errors="coerce")

    # Drop lignes invalides (ex: header lu comme data, lignes vides)
    df = df.dropna(subset=["year", "nh3_consumption_t"])

    # Types finaux
    df["year"] = df["year"].astype(int)
    df["nh3_consumption_t"] = df["nh3_consumption_t"].astype(float)

    return df


df_in = read_ammonia_raw(INPUT_CSV)

# Sélection pays
if CFG["input"]["countries"] == "ALL":
    COUNTRIES = sorted(df_in.loc[df_in["year"] == BASE_YEAR, "country"].unique().tolist())
else:
    COUNTRIES = [c.upper() for c in CFG["input"]["countries"]]

# Base 2019 : une ligne par (country, base_scenario)
df_base = df_in.loc[
    (df_in["country"].isin(COUNTRIES)) &
    (df_in["year"] == BASE_YEAR)
].copy()

if df_base.empty:
    raise ValueError(f"Aucune ligne trouvée pour l'année base_year={BASE_YEAR}.")

df_base = (
    df_base.sort_values(["country", "base_scenario"])
           .groupby(["country", "base_scenario"], as_index=False)
           .head(1)
)

print(f"{len(COUNTRIES)} pays détectés à {BASE_YEAR}.")
df_base.head()

27 pays détectés à 2019.


,country,base_scenario,year,nh3_consumption_t
1,AT,DE,2019,455000.0
5,AT,GA,2019,455000.0
9,BE,DE,2019,870000.0
13,BE,GA,2019,870000.0
17,BG,DE,2019,230000.0


In [39]:
base_by_key = {
    (r["country"], r["base_scenario"]): float(r["nh3_consumption_t"])
    for _, r in df_base.iterrows()
}
BASE_KEYS = sorted(base_by_key.keys())

len(BASE_KEYS), BASE_KEYS[:10]

(54,
 [('AT', 'DE'),
  ('AT', 'GA'),
  ('BE', 'DE'),
  ('BE', 'GA'),
  ('BG', 'DE'),
  ('BG', 'GA'),
  ('CY', 'DE'),
  ('CY', 'GA'),
  ('CZ', 'DE'),
  ('CZ', 'GA')])

In [40]:
def apply_cagr(value_base: float, years: np.ndarray, cagr: float, base_year: int) -> np.ndarray:
    """
    value(t) = value_base * (1 + cagr)^(t - base_year)
    """
    dt = years - base_year
    return value_base * np.power(1.0 + cagr, dt)


def maritime_nh3_series_total(years: np.ndarray) -> np.ndarray:
    """
    Trajectoire totale UE de demande maritime NH3 (t/an).
    """
    if not MAR_ENABLED:
        return np.zeros_like(years, dtype=float)
    return apply_cagr(MAR_BASE_2019_T, years, cagr=MAR_CAGR, base_year=BASE_YEAR)


def maritime_allocation_weights(base_keys: list[tuple[str, str]]) -> dict[tuple[str, str], float]:
    """
    Renvoie des poids w_k >= 0 qui somment à 1 sur les base_keys.
    - proportional_to_base : w_k ∝ conso NH3 2019 (base_by_key)
    - equal_by_country : même poids par pays (répliqué sur base_scenario si plusieurs)
    """
    if not MAR_ENABLED:
        return {k: 0.0 for k in base_keys}

    if len(base_keys) == 0:
        raise ValueError("base_keys vide: impossible d'allouer la demande maritime.")

    if MAR_ALLOC == "proportional_to_base":
        vals = np.array([base_by_key[k] for k in base_keys], dtype=float)
        tot = vals.sum()
        if tot <= 0:
            # fallback : égalitaire
            w = np.full(len(base_keys), 1.0 / len(base_keys), dtype=float)
        else:
            w = vals / tot
        return {k: float(w[i]) for i, k in enumerate(base_keys)}

    # equal_by_country
    countries = sorted({c for (c, _) in base_keys})
    if len(countries) == 0:
        raise ValueError("Aucun pays dans base_keys: allocation maritime impossible.")

    w_country = {c: 1.0 / len(countries) for c in countries}

    # si un pays a plusieurs base_scenario, on répartit également entre eux
    by_country: dict[str, list[tuple[str, str]]] = {}
    for k in base_keys:
        by_country.setdefault(k[0], []).append(k)

    weights: dict[tuple[str, str], float] = {}
    for c, ks in by_country.items():
        per_key = w_country[c] / len(ks)
        for k in ks:
            weights[k] = per_key

    # Sanity: somme des poids ~= 1
    s = float(sum(weights.values()))
    if abs(s - 1.0) > 1e-9:
        raise ValueError(f"Somme des poids maritime != 1 (={s}).")

    return weights


# Trajectoire totale UE (indépendante des exclusions pays par scénario)
MAR_TOTAL = maritime_nh3_series_total(YEARS)
MAR_TOTAL[:5]

array([0., 0., 0., 0., 0.])

In [43]:
def build_ammonia_scenario(tech_scenario: str, p: dict) -> pd.DataFrame:
    """
    - nh3_consumption_t : constant = 2019
    - domestic_share(t) : plateaux 5 ans de domestic_share_2019 -> domestic_share_2050
    - import = conso - domestic
    - decarb : plateaux 5 ans sur part h2_to_nh3 (0 -> h2_route_share_2050)
      => ch4_to_h2_to_nh3 = le complément sur le domestique
    - H2 réseau = nh3_domestic_h2_route * 0.18
    - H2 onsite (SMR) = nh3_domestic_ch4_route * 0.18
    - CH4 demande = H2 onsite * (tCH4 / tH2)
    - HB électricité = NH3 domestique * 0.98522 (convention)
    """
    # Filtrage des pays pour ce scénario
    keys = BASE_KEYS
    mar_weights = maritime_allocation_weights(keys)
    dom_2019 = float(p["domestic_share_2019"])
    dom_2050 = float(p["domestic_share_2050"])
    if not (0.0 <= dom_2019 <= 1.0 and 0.0 <= dom_2050 <= 1.0):
        raise ValueError(f"domestic_share doit être dans [0,1] pour {tech_scenario}.")

    decarb_start = int(p["decarb_start"])
    decarb_end = int(p["decarb_end"])
    h2_share_2050 = float(p["h2_route_share_2050"])
    if not (0.0 <= h2_share_2050 <= 1.0):
        raise ValueError(f"h2_route_share_2050 doit être dans [0,1] pour {tech_scenario}.")

    # Domestic share en plateaux
    domestic_share = plateau_series(
        YEARS, start_year=YEAR_START, end_year=YEAR_END,
        start_value=dom_2019, end_value=dom_2050,
        step_years=STEP_YEARS
    )
    domestic_share = np.clip(domestic_share, 0.0, 1.0)

    # Part H2-route sur le domestique (décarbonation) en plateaux
    h2_route_share = plateau_series(
        YEARS, start_year=decarb_start, end_year=decarb_end,
        start_value=0.0, end_value=h2_share_2050,
        step_years=STEP_YEARS
    )
    h2_route_share = np.clip(h2_route_share, 0.0, 1.0)

    out = []
    for (country, base_sc) in keys:
        cons0 = base_by_key[(country, base_sc)]
        # consommation “core” (ancrée 2019)
        nh3_core = np.full_like(YEARS, cons0, dtype=float)

        # add-on maritime réparti par poids
        w = mar_weights.get((country, base_sc), 0.0)
        nh3_maritime = MAR_TOTAL * w

        # consommation totale
        nh3_cons = nh3_core + nh3_maritime

        # Countries where imports are forbidden in this scenario
        no_imp = {str(c).strip().upper() for c in (p.get("no_import_countries", []) or [])}

        # --- domestic/import : construit pour garantir conso = domestic + import ---
        if country in no_imp:
            # Import interdit => tout est domestique
            nh3_domestic = nh3_cons
        else:
            # Domestic selon trajectoire (plateaux)
            nh3_domestic = nh3_cons * domestic_share

        # Clips de sécurité
        nh3_domestic = np.clip(nh3_domestic, 0.0, nh3_cons)

        # Import = résiduel (garantit l'adéquation)
        nh3_imported = nh3_cons - nh3_domestic
        nh3_imported = np.clip(nh3_imported, 0.0, None)

        # Split techno sur domestique (complémentarité exacte)
        nh3_domestic_h2_route = nh3_domestic * h2_route_share
        nh3_domestic_h2_route = np.clip(nh3_domestic_h2_route, 0.0, nh3_domestic)

        nh3_domestic_ch4_route = nh3_domestic - nh3_domestic_h2_route
        nh3_domestic_ch4_route = np.clip(nh3_domestic_ch4_route, 0.0, None)

        # H2
        h2_demand_network_t = nh3_domestic_h2_route * H2_T_PER_T
        h2_produced_onsite_t = nh3_domestic_ch4_route * H2_T_PER_T

        # CH4 pour produire l'H2 onsite
        ch4_demand_for_h2_t = h2_produced_onsite_t * CH4_T_PER_T_H2
        ch4_demand_for_nh3_t = ch4_demand_for_h2_t

        # Electricité HB (domestique uniquement)
        hb_electricity_kwh = nh3_domestic * HB_KWH_PER_T

        df_k = pd.DataFrame({
            "country": country,
            "base_scenario": base_sc,
            "scenario": tech_scenario,
            "year": YEARS,

            "nh3_consumption_t": nh3_cons,
            "nh3_domestic_t": nh3_domestic,
            "nh3_imported_t": nh3_imported,

            "nh3_domestic_ch4_to_h2_to_nh3_t": nh3_domestic_ch4_route,
            "nh3_domestic_h2_to_nh3_t": nh3_domestic_h2_route,

            "h2_demand_network_t": h2_demand_network_t,
            "h2_produced_onsite_t": h2_produced_onsite_t,

            "nh3_core_t": nh3_core,
            "nh3_maritime_t": nh3_maritime,

            "ch4_demand_for_h2_t": ch4_demand_for_h2_t,
            "ch4_demand_for_nh3_t": ch4_demand_for_nh3_t,

            "hb_electricity_kwh": hb_electricity_kwh,

            # Pour audit : shares appliqués
            "domestic_share": domestic_share,
            "h2_route_share_of_domestic": h2_route_share,
        })
        out.append(df_k)

    df = pd.concat(out, ignore_index=True)

    # Contrôles
    assert_non_negative(df, [
        "nh3_consumption_t", "nh3_domestic_t", "nh3_imported_t",
        "nh3_domestic_ch4_to_h2_to_nh3_t", "nh3_domestic_h2_to_nh3_t",
        "h2_demand_network_t", "h2_produced_onsite_t",
        "ch4_demand_for_h2_t", "ch4_demand_for_nh3_t",
        "hb_electricity_kwh",
        "domestic_share", "h2_route_share_of_domestic",
    ])

    # Identités d'adéquation
    err0 = (df["nh3_consumption_t"] - (df["nh3_core_t"] + df["nh3_maritime_t"])).abs().max()
    if err0 > 1e-6:
        raise ValueError(f"Incohérence NH3: total = core + maritime (max err={err0}).")
    err1 = (df["nh3_consumption_t"] - (df["nh3_domestic_t"] + df["nh3_imported_t"])).abs().max()
    if err1 > 1e-6:
        raise ValueError(f"Incohérence NH3: conso = domestic + import (max err={err1}).")

    err2 = (df["nh3_domestic_t"] - (df["nh3_domestic_ch4_to_h2_to_nh3_t"] + df["nh3_domestic_h2_to_nh3_t"])).abs().max()
    if err2 > 1e-6:
        raise ValueError(f"Incohérence NH3: domestic = ch4_route + h2_route (max err={err2}).")

    check_coverage(df, YEAR_START, YEAR_END)
    return df

In [44]:
def write_outputs(tech_scenario: str, df: pd.DataFrame) -> None:
    out_dir = SCENARIO_DIRS[tech_scenario]
    out_dir.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_dir / "ammonia.csv", index=False)


OUT_BASE.mkdir(parents=True, exist_ok=True)

all_outputs: dict[str, pd.DataFrame] = {}

for tech_scenario, p in SC_PARAMS.items():
    df_scn = build_ammonia_scenario(tech_scenario, p)
    write_outputs(tech_scenario, df_scn)
    all_outputs[tech_scenario] = df_scn

with (OUT_BASE / "config_effective.yaml").open("w", encoding="utf-8") as f:
    yaml.safe_dump(CFG, f, sort_keys=False, allow_unicode=True)

print("OK: scénarios NH3 générés dans", OUT_BASE)

OK: scénarios NH3 générés dans /Users/simonbrigode/Desktop/tp_pommes_kraft/data_country/industry/pommes-h2-network/data-pommes/scenarios-ammonia


In [23]:
all_outputs["full-eu-nh3"].head(12)

,country,base_scenario,scenario,year,nh3_consumption_t,nh3_domestic_t,nh3_imported_t,nh3_domestic_ch4_to_h2_to_nh3_t,nh3_domestic_h2_to_nh3_t,h2_demand_network_t,h2_produced_onsite_t,nh3_core_t,nh3_maritime_t,ch4_demand_for_h2_t,ch4_demand_for_nh3_t,hb_electricity_kwh,domestic_share,h2_route_share_of_domestic
0,AT,DE,full-eu-nh3,2019,455000.0,455000.0,0.0,455000.0,0.0,0.0,81900.0,455000.0,0.0,215526.315789,215526.315789,448275.1,1.0,0.0
1,AT,DE,full-eu-nh3,2020,455000.0,455000.0,0.0,455000.0,0.0,0.0,81900.0,455000.0,0.0,215526.315789,215526.315789,448275.1,1.0,0.0
2,AT,DE,full-eu-nh3,2021,455000.0,455000.0,0.0,455000.0,0.0,0.0,81900.0,455000.0,0.0,215526.315789,215526.315789,448275.1,1.0,0.0
3,AT,DE,full-eu-nh3,2022,455000.0,455000.0,0.0,455000.0,0.0,0.0,81900.0,455000.0,0.0,215526.315789,215526.315789,448275.1,1.0,0.0
4,AT,DE,full-eu-nh3,2023,455000.0,455000.0,0.0,455000.0,0.0,0.0,81900.0,455000.0,0.0,215526.315789,215526.315789,448275.1,1.0,0.0
5,AT,DE,full-eu-nh3,2024,455000.0,455000.0,0.0,455000.0,0.0,0.0,81900.0,455000.0,0.0,215526.315789,215526.315789,448275.1,1.0,0.0
6,AT,DE,full-eu-nh3,2025,455000.0,455000.0,0.0,455000.0,0.0,0.0,81900.0,455000.0,0.0,215526.315789,215526.315789,448275.1,1.0,0.0
7,AT,DE,full-eu-nh3,2026,455000.0,455000.0,0.0,455000.0,0.0,0.0,81900.0,455000.0,0.0,215526.315789,215526.315789,448275.1,1.0,0.0
8,AT,DE,full-eu-nh3,2027,455000.0,455000.0,0.0,455000.0,0.0,0.0,81900.0,455000.0,0.0,215526.315789,215526.315789,448275.1,1.0,0.0
9,AT,DE,full-eu-nh3,2028,455000.0,455000.0,0.0,455000.0,0.0,0.0,81900.0,455000.0,0.0,215526.315789,215526.315789,448275.1,1.0,0.0


In [24]:
# Contrôle agrégé UE par année, pour vérifier les rampes :
df_agg = (
    all_outputs["intermediary-eu-nh3"]
    .groupby("year", as_index=False)[
        ["nh3_consumption_t", "nh3_imported_t", "nh3_domestic_t",
         "h2_demand_network_t", "ch4_demand_for_h2_t"]
    ]
    .sum()
)

df_agg.head(10)

,year,nh3_consumption_t,nh3_imported_t,nh3_domestic_t,h2_demand_network_t,ch4_demand_for_h2_t
0,2019,23370000.0,0.000000,2.337000e+07,0.000000,1.107000e+07
1,2020,23370000.0,0.000000,2.337000e+07,0.000000,1.107000e+07
2,2021,23370000.0,0.000000,2.337000e+07,0.000000,1.107000e+07
3,2022,23370000.0,0.000000,2.337000e+07,0.000000,1.107000e+07
4,2023,23370000.0,0.000000,2.337000e+07,0.000000,1.107000e+07
5,2024,23370000.0,500785.714286,2.286921e+07,0.000000,1.083279e+07
6,2025,23370000.0,500785.714286,2.286921e+07,686076.428571,9.027321e+06
7,2026,23370000.0,500785.714286,2.286921e+07,686076.428571,9.027321e+06
8,2027,23370000.0,500785.714286,2.286921e+07,686076.428571,9.027321e+06
9,2028,23370000.0,500785.714286,2.286921e+07,686076.428571,9.027321e+06
